In [1]:
import advent
data = advent.get_lines(10, map_fn = lambda x: x.split(' '))

In [2]:
from typing import NamedTuple, Iterable
from itertools import chain, combinations

def to_int_list(s: str) -> list[int]:
    return [int(i) for i in s[1:-1].split(',')]

def powerset(lst: list) -> Iterable[tuple]:
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    return chain.from_iterable(combinations(lst, r) for r in range(len(lst)+1))


class Lights():
    target: list[bool]
    lights: list[bool]

    def __init__(self, target: list[bool], lights: list[bool]):
        self.target = target
        self.lights = lights

    def __str__(self) -> str:
        return ''.join('#' if l else '.' for l in self.lights) + ' -> ' + ''.join('#' if t else '.' for t in self.target)

    def is_done(self) -> bool:
        return self.lights == self.target
    
    def toggle(self, lights):
        for i in lights:
            self.lights[i] = not self.lights[i]

    def toggle_multiple(self, lights_list: list[list[int]], which: list[int]):
        for i in which:
            self.toggle(lights_list[i])
        return self.is_done()
    
    def reset(self):
        self.lights = [False] * len(self.target)

    @staticmethod
    def from_str(target: str):
        target = [c == '#' for c in target[1:-1]]
        lights = [False] * len(target)
        return Lights(target=target, lights=lights)
    
str(Lights.from_str('[##.....#.]'))

'......... -> ##.....#.'

In [3]:
def find_record(line):
    lights = Lights.from_str(line[0])
    toggles = [to_int_list(s) for s in line[1:-1]]

    record = 99999
    for which in powerset(list(range(len(toggles)))):
        lights.reset()
        if lights.toggle_multiple(toggles, which):
            record = min(record, len(which))
    return record

result = sum([find_record(line) for line in data])
print(result)


558


In [4]:
# Part 2

In [7]:
from advent.maze import solve_maze_no_tqdm, solve_maze

class Lights2(NamedTuple):
    target: tuple[int]
    lights: tuple[int]

    def is_done(self) -> bool:
        return self.lights == self.target
    
    def toggle(self, lights) -> 'Lights2':
        new_lights = list(self.lights)
        for i in lights:
            new_lights[i] = new_lights[i] + 1
            if new_lights[i] > self.target[i]:
                return None
        return Lights2(target=self.target, lights=tuple(new_lights))
    
    def h(self):
        return sum(self.target) - sum(self.lights)

    @staticmethod
    def from_str(target: str):
        target = to_int_list(target)
        lights = [0] * len(target)
        return Lights2(target=tuple(target), lights=tuple(lights))

def adjacent(state: Lights2, lights_list: list[list[int]]) -> Iterable[tuple[Lights2, int]]:
    for i in range(len(lights_list)):
        adj = state.toggle(lights_list[i])
        if adj is not None:
            yield (adj, 1)

start = Lights2.from_str('{3,5,4,7}')
lights_list = [(3,), (1,3), (2,), (2,3), (0,2), (0,1)]

shortest_path = solve_maze_no_tqdm(start, lambda state: state.is_done(), lambda state: adjacent(state, lights_list), lambda state: state.h())
assert shortest_path[0] == 10

In [ ]:
result = 0
for line in [data[0]]:
    start = Lights2.from_str(line[-1])
    toggles = [to_int_list(s) for s in line[1:-1]]
    shortest_path = solve_maze(start, lambda state: state.is_done(), lambda state: adjacent(state, toggles), lambda state: state.h())
    result += shortest_path[0]
print(result)